# Create calculation schemes for VASP

Create defined VASP calculation schemes using the `Schemes` class in the `vasp.schemes` module. This class contains pre-defined schemes to perform different types of calculations. The starting point is a `Structure` object and `Incar` and `Kpoints` objects from `pymatgen`. These will define the basis of inputs for the scheme, where only the relavant INCAR tags and KPOINTS (where needed) will be changed according to the different scheme steps. Only the `Structure` argument is needed, if all other arguments are missing the objects will be constructed according to the `DefaultInputs` class.

### Define structure

We will retrieve the structure of Silicon from the Materials Project database using `pymatgen`.

In [1]:
from pynter.tools.materials_project import MPDatabase

structure = MPDatabase('mp-149').get_structure()
print(structure)

Full Formula (Si2)
Reduced Formula: Si
abc   :   3.839943   3.839943   3.839943
angles:  60.000000  60.000000  60.000000
Sites (2)
  #  SP        a      b      c
---  ----  -----  -----  -----
  0  Si    0.875  0.875  0.875
  1  Si    0.125  0.125  0.125


### Define set of inputs 

now we need to define a set of common inputs for the calculations. In this case we explicitely use the `DefaultInputs` class to show how it works, but if the default inputs are needed one can just omit the arguments when defining the `Schemes` class. 

In [4]:
from pynter.vasp.default_inputs import DefaultInputs
from pynter.vasp.schemes import Schemes

di = DefaultInputs(structure)
incar_settings = di.get_incar_default(xc='PBE')
kpoints = di.get_kpoints_default(kppa=1000)
# Get Pymatgen Potcar object. The PMG_VASP_PSP directory has to be defined in .pmgrc.yaml file as required by Pymatgen.
potcar = di.get_potcar(potcar_functional='PBE')

print(incar_settings)
print(kpoints)

{'IBRION': 2, 'NSW': 0, 'ISIF': 2, 'EDIFFG': -0.05, 'ISPIN': 1, 'LWAVE': '.TRUE.', 'LCHARG': '.TRUE.', 'LORBIT': 10, 'LVTOT': '.TRUE.', 'ENCUT': 550, 'EDIFF': 1e-06, 'ISMEAR': 0, 'SIGMA': 0.05, 'NELM': 200, 'ALGO': 'Normal', 'AMIX': 0.2, 'LREAL': '.FALSE.', 'SYSTEM': 'Si', '#### Default PBE: system': 'Si', 'ISYM': 2}
pymatgen 4.7.6+ generated KPOINTS with grid density = 1000 / atom
0
Gamma
8 8 8



### Create Schemes object

Now we can create the `Schemes` object from the previously defined inputs. The argument `job_settings` is None by default, which means that the default settings for the job script are retrieved from the `config.yml` file when the input files are written.

In [11]:
# define path
import os
homedir = os.getenv("HOME")
path = os.path.join(homedir,'pynter-tutorials-files','Si-BS')

In [12]:
cs = Schemes(path,structure=structure,incar_settings=incar_settings,kpoints=kpoints,
             potcar=potcar,job_settings=None,name='Si')

### Create list of Job objects

Now we can use on of the methods of this class to select the desired scheme and create a list of `VaspJob` objects of the `pymatgen.data.jobs` module. As an example we pick the `pbe_electronic_structure` scheme, which is made by a first electronic minimization step, followed by a DOS calculation with the number of kpoints tripled by default in every direction, and a non-sc BS calculation, with k-path build according to the defaults paths in Pymatgen.

In [13]:
jobs = cs.pbe_electronic_structure();
jobs

[Job "Si_PBE-el-str_1", Job "Si_PBE-el-str_2", Job "Si_PBE-el-str_3"]

In [14]:
# Check input params
for job in jobs:
    print(job.name+':\n')
    print(job.incar)
    print(job.kpoints)
    print(job.job_settings)
    print('')

Si_PBE-el-str_1:

#### Default PBE: system = Si
ALGO = Normal
AMIX = 0.2
EDIFF = 1e-06
EDIFFG = -0.05
ENCUT = 550
IBRION = 2
ISIF = 3
ISMEAR = 0
ISPIN = 1
ISYM = 2
LCHARG = .TRUE.
LORBIT = 10
LREAL = .FALSE.
LVTOT = .TRUE.
LWAVE = .TRUE.
NELM = 200
NSW = 100
SIGMA = 0.05
SYSTEM = Si

pymatgen 4.7.6+ generated KPOINTS with grid density = 1000 / atom
0
Gamma
8 8 8

{'name': 'Si_PBE-el-str_1'}

Si_PBE-el-str_2:

#### Default PBE: system = Si
ALGO = Normal
AMIX = 0.2
EDIFF = 1e-06
EDIFFG = -0.05
ENCUT = 550
IBRION = 2
ICHARG = 1
ISIF = 3
ISMEAR = -5
ISPIN = 1
ISTART = 1
ISYM = 2
LCHARG = .TRUE.
LORBIT = 10
LREAL = .FALSE.
LVTOT = .TRUE.
LWAVE = .TRUE.
NEDOS = 2000
NELM = 200
NSW = 0
SIGMA = 0.05
SYSTEM = Si

Default gamma
0
Gamma
24 24 24

{'name': 'Si_PBE-el-str_2'}

Si_PBE-el-str_3:

#### Default PBE: system = Si
ALGO = Normal
AMIX = 0.2
EDIFF = 1e-06
EDIFFG = -0.05
ENCUT = 550
IBRION = 2
ICHARG = 11
ISIF = 3
ISMEAR = 0
ISPIN = 1
ISTART = 1
ISYM = 2
LCHARG = .TRUE.
LORBIT = 11
LREAL = .F

### Write files

Now we can create input files with the `write_input` method in the `VaspJob` class. Alternatively we can create a `Dataset` from the `data.datasets` to organize the jobs in a group.

In [15]:
from pynter.data.datasets import Dataset

ds = Dataset(jobs)
ds

                formula        group nodes is_converged
job_name                                               
Si_PBE-el-str_1     Si2  1-PBE-relax               None
Si_PBE-el-str_2     Si2    2-PBE-DOS               None
Si_PBE-el-str_3     Si2     3-PBE-BS               None

In [16]:
ds.write_jobs_input()